In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.4/640.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 7.2 MB/s eta 0:00:

In [54]:
# Make sure you have a GPU running
!nvidia-smi

Mon Mar  6 07:03:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    32W /  70W |   2621MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [55]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest main of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-brwbo0vf/farm-haystack_1718f2a44aea463a92c614c4af723c9e
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-brwbo0vf/farm-haystack_1718f2a44aea463a92c614c4af723c9e
  Resolved https://github.com/deepset-ai/haystack.git to commit 1a421669786c80feee5ef17511202529c9c30c84
  Installing build dependencies ... done
  Getting requirements to

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [38]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Prepare the document: Add DocOps Document here

In [39]:
path_to_zip_file = 'drive/My Drive/MF/'
directory_to_extract_to = 'Opsb_Corpus'
doc_dir = path_to_zip_file+'/'+directory_to_extract_to



In [40]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=500,
    split_overlap=50,
    split_respect_sentence_boundary=True,
)

In [41]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
#indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["TextConverter"])

In [42]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/499 [00:00<?, ?it/s]

Updating BM25 representation...:   0%|          | 0/499 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': 'Get started\nContainerized Operations Bridge offers a monitoring solution that consolidates data from existing tools. It applies automated\ndiscovery, monitoring, analytics, and remediation to data across traditional, private, public, multi cloud, and container based\ninfrastructure. Using this solution, you can achieve the following goals:\nEvent consolidation - You can manage events from all your domains by consolidating them in Operations Bridge Manager\n(OBM). This consolidated view helps your IT operators to identify the causes of IT incidents and rectify issues. You can also\ncorrelate similar events, suppress unwanted events, or promote events based on your business requirements. \nNoise reduction - You can also get automatically correlated events in OBM. Driven by artificial intelligence (AI), automatic\ncorrelation reduces event related noise.\nCross domain business service driven out-of-the-box Reports. You can collect metrics, topology,

In [43]:
#search system will use a Retriever, so we need to initialize it. 
#A Retriever sifts through all the Documents and returns only the ones relevant to the question. 
#This tutorial uses the BM25 algorithm

from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [44]:
#Initializing the Reader
#A Reader scans the texts it received from the Retriever and extracts the top answer candidates. 

from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [45]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

The pipeline’s ready, you can now go ahead and ask a question!

In [46]:
prediction = querying_pipeline.run(
    query="Tell me about Opsb event correlation ?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]

In [47]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Automatic Event Correlation', 'type': 'extractive', 'score': 0.26186585426330566, 'context': 'holder Dashboards (Business Value Dashboard)\nOPTIC Reporting \nAutomatic Event Correlation\nAgentless Monitoring*\nHyperscale Observability*\nContainerize', 'offsets_in_document': [{'start': 166, 'end': 193}], 'offsets_in_context': [{'start': 62, 'end': 89}], 'document_ids': ['5fc52982237960a004fc3919c6add461'], 'meta': {}}>,
             <Answer {'answer': 'Automatic', 'type': 'extractive', 'score': 0.23116007447242737, 'context': 'es:\nStakeholder Dashboards (Business Value Dashboard)\nOPTIC Reporting \nAutomatic Event Correlation\nAgentless Monitoring*\nHyperscale Observability*\nCon', 'offsets_in_document': [{'start': 11368, 'end': 11377}], 'offsets_in_context': [{'start': 71, 'end': 80}], 'document_ids': ['5fc52982237960a004fc3919c6add461'], 'meta': {}}>,
             <Answer {'answer': 'Automatic Event Correlation\nAgentless Monitoring', 'type': 'extract

In [48]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="medium" ## Choose from `minimum`, `medium`, and `all`
)


Query: Tell me about Opsb event correlation ?
Answers:
[   {   'answer': 'Automatic Event Correlation',
        'context': 'holder Dashboards (Business Value Dashboard)\n'
                   'OPTIC Reporting \n'
                   'Automatic Event Correlation\n'
                   'Agentless Monitoring*\n'
                   'Hyperscale Observability*\n'
                   'Containerize',
        'score': 0.26186585426330566},
    {   'answer': 'Automatic',
        'context': 'es:\n'
                   'Stakeholder Dashboards (Business Value Dashboard)\n'
                   'OPTIC Reporting \n'
                   'Automatic Event Correlation\n'
                   'Agentless Monitoring*\n'
                   'Hyperscale Observability*\n'
                   'Con',
        'score': 0.23116007447242737},
    {   'answer': 'Automatic Event Correlation\nAgentless Monitoring',
        'context': 'boards (Business Value Dashboard)\n'
                   'OPTIC Reporting \n'
                   

In [49]:
def opsb_chatbot(querying_pipeline,user_query,top_k):
  prediction = querying_pipeline.run(
    query=user_query,
    params={
        "Retriever": {"top_k": top_k*2},
        "Reader": {"top_k": top_k}
    }
  )
  return prediction

In [50]:
from haystack.utils import print_answers
result = opsb_chatbot(querying_pipeline,"Tell me about OPTIC Reporting capability please",5)
print_answers(
    result,
    details="all" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]


Query: Tell me about OPTIC Reporting capability please
Answers:
[   <Answer {'answer': 'Automatic Event Correlation', 'type': 'extractive', 'score': 0.4425888955593109, 'context': 'holder Dashboards (Business Value Dashboard)\nOPTIC Reporting \nAutomatic Event Correlation\nAgentless Monitoring*\nHyperscale Observability*\nContainerize', 'offsets_in_document': [{'start': 11368, 'end': 11395}], 'offsets_in_context': [{'start': 62, 'end': 89}], 'document_ids': ['5fc52982237960a004fc3919c6add461'], 'meta': {}}>,
    <Answer {'answer': 'opticReporting', 'type': 'extractive', 'score': 0.4167580008506775, 'context': 'idge Suite offers. Users can enable one or multiple components.\n  # opticReporting: Reporting component contains OPTIC Data Lake, BVD and Collection S', 'offsets_in_document': [{'start': 798, 'end': 812}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_ids': ['62442ee3901a92250ac1714b7f89954b'], 'meta': {}}>,
    <Answer {'answer': 'Automatic Event Correlation', 't

In [51]:
from haystack.utils import print_answers
result = opsb_chatbot(querying_pipeline,"what is BYOBI)",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]


Query: what is BYOBI)
Answers:
[   <Answer {'answer': 'Bring your own Business Intelligence', 'type': 'extractive', 'score': 0.9251627922058105, 'context': 't \nSynthetic Transaction \nReal User Monitor \nApplication\nBring your own Business Intelligence (BYOBI)\nYou can use third-party Business Intelligence (B', 'offsets_in_document': [{'start': 2449, 'end': 2485}], 'offsets_in_context': [{'start': 57, 'end': 93}], 'document_ids': ['bcd422f5476478db186938529574d177'], 'meta': {}}>,
    <Answer {'answer': 'Bring your own Business Intelligence', 'type': 'extractive', 'score': 0.8157068490982056, 'context': 't \nSynthetic Transaction \nReal User Monitor \nApplication\nBring your own Business Intelligence (BYOBI)\nYou can use third-party Business Intelligence (B', 'offsets_in_document': [{'start': 876, 'end': 912}], 'offsets_in_context': [{'start': 57, 'end': 93}], 'document_ids': ['bcd422f5476478db186938529574d177'], 'meta': {}}>,
    <Answer {'answer': 'business intelligence tool', 'ty

In [52]:
from haystack.utils import print_answers
result = opsb_chatbot(querying_pipeline,"what is Stakeholder Dashboard)",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]


Query: what is Stakeholder Dashboard)
Answers:
[   <Answer {'answer': 'Business Value Dashboard', 'type': 'extractive', 'score': 0.74318528175354, 'context': 'reate a deployment plan for installing\nStakeholder Dashboards (Business Value Dashboard) on a single node in K3s environment, you must consider the fo', 'offsets_in_document': [{'start': 540, 'end': 564}], 'offsets_in_context': [{'start': 63, 'end': 87}], 'document_ids': ['755f845a45a1e269f828b79b68cf8983'], 'meta': {}}>,
    <Answer {'answer': 'gives you visual information of live data using tables, charts, and\nwidgets', 'type': 'extractive', 'score': 0.5250613689422607, 'context': 'Dashboard\n: The stakeholder dashboard gives you visual information of live data using tables, charts, and\nwidgets. Real-time data can be streamed from', 'offsets_in_document': [{'start': 703, 'end': 778}], 'offsets_in_context': [{'start': 38, 'end': 113}], 'document_ids': ['9e9ce6a04a1e87c3b832a726c62bd185'], 'meta': {}}>,
    <Answer {'answer':

In [53]:
result = opsb_chatbot(querying_pipeline,"Agentless Monitoring License requirement ?)",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]


Query: Agentless Monitoring License requirement ?)
Answers:
[   <Answer {'answer': 'Actions', 'type': 'extractive', 'score': 0.11176839470863342, 'context': 'd configuration item topology for infrastructure and applications.\n5\n. \nActions\n: Actions you can perform on a monitor: Edit, Copy, Alerts, and Delete', 'offsets_in_document': [{'start': 1814, 'end': 1821}], 'offsets_in_context': [{'start': 72, 'end': 79}], 'document_ids': ['2f5f43fb62e7c095563a3fc382d94601'], 'meta': {}}>,
    <Answer {'answer': 'Sitescope', 'type': 'extractive', 'score': 0.07851079106330872, 'context': "ess Monitoring, you don't need to apply for an additional license. The Sitescope license\nenables you to access Agentless Monitoring.\nContainerized Ope", 'offsets_in_document': [{'start': 3210, 'end': 3219}], 'offsets_in_context': [{'start': 71, 'end': 80}], 'document_ids': ['2dd94f007a54e3f6f0beedd2f0ed5002'], 'meta': {}}>,
    <Answer {'answer': 'Express (Agentless Monitoring user interface only)\nPremium